In [1]:
reset -fs

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
import string
import os

#import nltk
#from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA

# Read in data

In [3]:
modern_love_df = pd.read_csv('modern_love_df.csv')

In [4]:
modern_love_df.shape

(1615, 12)

In [5]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   article_url             1615 non-null   object
 1   article                 1609 non-null   object
 2   article_id              1615 non-null   object
 3   article_headline        1615 non-null   object
 4   article_abstract        1615 non-null   object
 5   article_snippet         1615 non-null   object
 6   article_lead_paragraph  1604 non-null   object
 7   article_pub_date        1615 non-null   object
 8   article_news_desk       1615 non-null   object
 9   article_section         1615 non-null   object
 10  article_type_material   1615 non-null   object
 11  article_word_count      1615 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 151.5+ KB


# Clean data

In [6]:
pd.set_option("display.max_colwidth", 1)

In [7]:
# drop rows with na in article column

modern_love_df = modern_love_df.dropna(subset=['article'])

In [8]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   article_url             1609 non-null   object
 1   article                 1609 non-null   object
 2   article_id              1609 non-null   object
 3   article_headline        1609 non-null   object
 4   article_abstract        1609 non-null   object
 5   article_snippet         1609 non-null   object
 6   article_lead_paragraph  1599 non-null   object
 7   article_pub_date        1609 non-null   object
 8   article_news_desk       1609 non-null   object
 9   article_section         1609 non-null   object
 10  article_type_material   1609 non-null   object
 11  article_word_count      1609 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 163.4+ KB


In [9]:
# rename article column

modern_love_df = modern_love_df.rename({'article':'essay'}, axis=1)

In [10]:
# rename the rest of the columns by removing 'article_'

modern_love_df.columns = modern_love_df.columns.str.replace('article_',"")

In [11]:
# check for duplicated rows

modern_love_df.duplicated().sum() # there are none!

0

In [12]:
# convert pub_date to datetime and then keep only the date

modern_love_df['pub_date'] = pd.to_datetime(modern_love_df['pub_date']).dt.date

In [13]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   url             1609 non-null   object
 1   essay           1609 non-null   object
 2   id              1609 non-null   object
 3   headline        1609 non-null   object
 4   abstract        1609 non-null   object
 5   snippet         1609 non-null   object
 6   lead_paragraph  1599 non-null   object
 7   pub_date        1609 non-null   object
 8   news_desk       1609 non-null   object
 9   section         1609 non-null   object
 10  type_material   1609 non-null   object
 11  word_count      1609 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 163.4+ KB


In [14]:
# see how many articles mention the nytimes in the essay, as this text should be removed

len(modern_love_df[modern_love_df.essay.str.contains('nytimes.com|New York Times')])

105

In [15]:
# drop these essays for times' sake (go back to data acquisition stage to remove these paragraphs, time permitting)

modern_love_df_rev = modern_love_df[~modern_love_df.essay.str.contains('nytimes.com|New York Times')]

In [16]:
# see how many articles mention 'This essay is part of a collaborative project with Black History, Continued. We invited readers and renowned writers to respond to the question “What is Black love today?”'

len(modern_love_df_rev[modern_love_df_rev.essay.str.contains('collaborative project')])

8

In [17]:
modern_love_df_rev[modern_love_df_rev.essay.str.contains('collaborative project')]

url  \
2   https://www.nytimes.com/2022/02/11/style/modern-love-gay-couple.html                       
3   https://www.nytimes.com/2022/02/11/style/modern-love-confidence-self-love.html             
4   https://www.nytimes.com/2022/02/11/style/modern-love-black-boyfriend-applications.html     
5   https://www.nytimes.com/2022/02/11/style/modern-love-queer-dating-open-relationship.html   
7   https://www.nytimes.com/2022/02/11/style/modern-love-bond-father.html                      
8   https://www.nytimes.com/2022/02/11/style/modern-love-black-boys.html                       
9   https://www.nytimes.com/2022/02/11/style/modern-love-healer-heartbreak.html                
10  https://www.nytimes.com/2022/02/11/style/modern-love-teeth-smile.html                      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [18]:
# remove opening sentences from collaborative project with Black History, Continued

sentences = 'This essay is part of a collaborative project with Black History, Continued. We invited readers and renowned writers to respond to the question “What is Black love today?” '
print(len(sentences))
sentences[:172]
collaborative_project = 'collaborative project'
sentences_to_remove = lambda x: x[172:] if collaborative_project in x else x
modern_love_df_rev['essay'] = modern_love_df['essay'].map(sentences_to_remove)

172


<ipython-input-18-b77f0ab32bac>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay'] = modern_love_df['essay'].map(sentences_to_remove)


# Topic modeling utility

Before creating a baseline model, create a pipeline function that does the following:

- Arguments are the corpus, vectorizer and topic_modeler
- Applies these in order
- Prints the top 15 words for each topic found from training

In [125]:
def make_topics(corpus, vectorizer, topic_modeler, print_n_words=15):
    """A very simple pipeline."""
    
    # Vectorize documents into a document-word matrix.
    doc_word_vectors = vectorizer.fit_transform(corpus)
    
    # Fit the topic model.
    doc_topic_vectors = topic_modeler.fit_transform(doc_word_vectors)
    
    # Print the topics.
    vocab = vectorizer.get_feature_names()
    for idx, topic in enumerate(topic_modeler.components_):
        # Select the top 15 words in vocab for this topic.
        top_words = [vocab[i].upper() for i in topic.argsort()[:-print_n_words-1:-1]]
        print(f"Topic {idx}:\n", ", ".join(top_words), "\n")
    
    return doc_topic_vectors

# Make a baseline topic model

As priors, use Tfidf vectorizer and NMF topic modeler

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [150]:
corpus = modern_love_df_rev['essay']
vectorizer = TfidfVectorizer()
topic_modeler = NMF(5, random_state=42, max_iter=1000) # start with 5 topics

make_topics(corpus, vectorizer, topic_modeler)

Topic 0:
 THE, AND, OF, TO, IN, WAS, THAT, MY, HAD, IT, AS, ON, WITH, FOR, AT 

Topic 1:
 SHE, HER, MY, WAS, TO, MOTHER, ME, AND, HAD, IN, FOR, WITH, WHEN, FATHER, GRANDMOTHER 

Topic 2:
 HE, HIS, HIM, MY, WAS, TO, ME, AND, HAD, SAID, WITH, BUT, FOR, IT, FATHER 

Topic 3:
 WE, OUR, AND, THE, IN, TO, US, TOGETHER, WERE, OTHER, EACH, ON, OF, FOR, YEARS 

Topic 4:
 YOU, TO, IT, THAT, YOUR, IS, LOVE, HAVE, ME, BE, BUT, SAID, NOT, ARE, AND 



array([[0.11507637, 0.        , 0.11676861, 0.04370471, 0.02735668],
       [0.10669412, 0.02223463, 0.06818305, 0.05797607, 0.00719347],
       [0.07581803, 0.15823511, 0.        , 0.0122531 , 0.02779558],
       ...,
       [0.        , 0.        , 0.05769031, 0.04955557, 0.        ],
       [0.03555944, 0.00725304, 0.        , 0.00149665, 0.04506553],
       [0.05243624, 0.00447371, 0.00288168, 0.00763272, 0.04180893]])

Looks like the topics are capturing lots of stop words - time to remove them!

# Topic Model 2 - remove stop words

Use spaCy package for text processing

In [109]:
import spacy
nlp = spacy.load('en_core_web_sm')
#from spacy.language import Language

In [110]:
docs = list(nlp.pipe(modern_love_df_rev['essay']))
modern_love_df_rev['spacy_essay'] = docs

<ipython-input-110-5ef2bef2e572>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['spacy_essay'] = docs


In [133]:
docs_no_sw = [[str(w) for w in doc if not w.is_stop] for doc in docs]
modern_love_df_rev['essay_no_stopwords'] = docs_no_sw

<ipython-input-133-2162efaf4326>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_no_stopwords'] = docs_no_sw


In [149]:
corpus = [token for doc in docs_no_sw for token in doc]
vectorizer = TfidfVectorizer()
topic_modeler = NMF(5, random_state=42, max_iter=1000)

make_topics(corpus, vectorizer, topic_modeler)

Topic 0:
 SAID, DAY, NEW, WAY, FELT, TOLD, WANTED, FATHER, LATER, ASKED, HOME, HUSBAND, PEOPLE, THOUGHT, KNEW 

Topic 1:
 TIME, DAY, NEW, TOLD, LATER, ASKED, HOME, HUSBAND, NIGHT, THOUGHT, YEAR, FAMILY, LONG, 3½, 2½ 

Topic 2:
 YEARS, DAY, NEW, WAY, TOLD, WANTED, ASKED, HOME, KNEW, THOUGHT, NIGHT, LONG, FAMILY, GOING, OLD 

Topic 3:
 LIFE, DAY, WAY, FATHER, ASKED, PEOPLE, THOUGHT, MAN, KNEW, YEAR, FAMILY, 3½, FRIENDS, 2½, GOING 

Topic 4:
 MOTHER, DAY, WAY, WANTED, TOLD, HOME, ASKED, LATER, HUSBAND, THOUGHT, KNEW, LONG, GOOD, AWAY, GOT 



array([[5.90690174e-60, 9.42748752e-44, 2.66349557e-36, 6.21628273e-32,
        5.95953483e-32],
       [5.82743839e-57, 3.53257698e-42, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

Things are looking better, but now it's time to remove non-alpha characters.

# Topic Model 3 - remove punctuation and non-alpha characters

In [135]:
docs_no_sw_alpha = [[str(w) for w in doc if w.is_alpha and \
              (not w.is_stop and not w.is_punct and not w.like_num)] for doc in docs]
modern_love_df_rev['essay_alpha'] = docs_no_sw_alpha

<ipython-input-135-98a12ec3e5bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_alpha'] = docs_no_sw_alpha


In [151]:
corpus = [token for doc in docs_no_sw_alpha for token in doc]
vectorizer = TfidfVectorizer()
topic_modeler = NMF(5, random_state=42, max_iter=1000)

make_topics(corpus, vectorizer, topic_modeler)

Topic 0:
 SAID, DAY, NEW, WAY, TOLD, FELT, WANTED, ASKED, HOME, FATHER, LATER, PEOPLE, HUSBAND, THOUGHT, KNEW 

Topic 1:
 TIME, TOLD, FELT, WANTED, LATER, HOME, PEOPLE, HUSBAND, THOUGHT, KNEW, MAN, CHILDREN, GOOD, GOING, AWAY 

Topic 2:
 YEARS, NEW, TOLD, WANTED, FELT, FATHER, HUSBAND, PEOPLE, THOUGHT, MAN, YEAR, CHILDREN, GOING, GOOD, AWAY 

Topic 3:
 LIFE, DAY, WAY, FELT, LATER, ASKED, NIGHT, MONTHS, OLD, LEFT, MET, TOOK, LITTLE, FOUND, HOUSE 

Topic 4:
 MOTHER, DAY, LATER, FATHER, PEOPLE, NIGHT, MAN, KNEW, THOUGHT, YEAR, FAMILY, FRIENDS, MONTHS, OLD, RIGHT 



array([[0.00000000e+00, 0.00000000e+00, 3.45410900e-40, 0.00000000e+00,
        0.00000000e+00],
       [1.63674132e-62, 7.62924334e-45, 0.00000000e+00, 2.42571973e-31,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

# Topic Model 4 - keep only nouns

In [146]:
docs_no_sw_alpha_nouns = [[str(w) for w in doc if w.is_alpha and \
                           (w.pos_ == 'NOUN' or w.pos_ == 'PROPN') and \
                           (not w.is_stop and not w.is_punct and not w.like_num)] \
                          for doc in docs]
modern_love_df_rev['essay_noun'] = docs_no_sw_alpha_nouns

<ipython-input-146-79a2c424c6fe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_noun'] = docs_no_sw_alpha_nouns


In [152]:
corpus = [token for doc in docs_no_sw_alpha_nouns for token in doc]
vectorizer = TfidfVectorizer()
topic_modeler = NMF(5, random_state=42, max_iter=1000)

make_topics(corpus, vectorizer, topic_modeler)

Topic 0:
 TIME, FATHER, WAY, HUSBAND, NIGHT, YEAR, PEOPLE, FRIENDS, FAMILY, MAN, MONTHS, CHILDREN, THINGS, MARRIAGE, FRIEND 

Topic 1:
 YEARS, WAY, FATHER, HUSBAND, FRIEND, PARENTS, SCHOOL, PERSON, BED, THING, NEW, WEEKS, SON, DAUGHTER, WIFE 

Topic 2:
 LIFE, WAY, FATHER, HUSBAND, PARENTS, FRIEND, HOUSE, SCHOOL, PERSON, BED, MEN, WORLD, SON, THING, WEEKS 

Topic 3:
 MOTHER, FATHER, WAY, HUSBAND, MAN, PEOPLE, YEAR, FAMILY, CHILDREN, HOUSE, THINGS, HOME, EYES, PERSON, BED 

Topic 4:
 DAY, PEOPLE, MAN, HUSBAND, YEAR, FAMILY, FRIENDS, CHILDREN, MONTHS, MARRIAGE, THINGS, HOUSE, EYES, HOME, PERSON 



array([[2.19369154e-18, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

# Topic Model 5 - lemmatize tokens

In [157]:
docs_no_sw_alpha_nouns_lemma = [[w.lemma_.lower() for w in doc if w.is_alpha and \
                           (w.pos_ == 'NOUN' or w.pos_ == 'PROPN') and \
                           (not w.is_stop and not w.is_punct and not w.like_num)] \
                          for doc in docs]
modern_love_df_rev['essay_lemma'] = docs_no_sw_alpha_nouns_lemma

<ipython-input-157-453f71608ca9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_lemma'] = docs_no_sw_alpha_nouns_lemma


In [158]:
corpus = [token for doc in docs_no_sw_alpha_nouns_lemma for token in doc]
vectorizer = TfidfVectorizer()
topic_modeler = NMF(5, random_state=42, max_iter=1000)

make_topics(corpus, vectorizer, topic_modeler)

Topic 0:
 YEAR, FRIEND, MAN, WAY, CHILD, FATHER, THING, NIGHT, PEOPLE, HUSBAND, MONTH, FAMILY, WOMAN, HAND, PARENT 

Topic 1:
 TIME, FRIEND, MAN, THING, FAMILY, HAND, SCHOOL, HOUSE, DATE, STORY, PERSON, DAUGHTER, PLACE, WORLD, FACE 

Topic 2:
 DAY, FRIEND, CHILD, THING, PEOPLE, WEEK, HAND, DATE, STORY, WORD, BED, DAUGHTER, PERSON, PLACE, FACE 

Topic 3:
 LIFE, MAN, FATHER, HUSBAND, FAMILY, WEEK, HOUSE, HOUR, DATE, WORD, STORY, WORLD, DAUGHTER, WIFE, GIRL 

Topic 4:
 MOTHER, WAY, FATHER, CHILD, NIGHT, HUSBAND, PEOPLE, MONTH, WOMAN, WEEK, PARENT, ROOM, MARRIAGE, SCHOOL, EYE 



array([[0.        , 0.        , 0.13542786, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       ...,
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

# Look at top most common and least common noun tokens to perhaps remove some

In [159]:
noun_tokens = [token for doc in modern_love_df_rev['essay_lemma'] for token in doc]
set_tokens = set(noun_tokens)
word_freq = Counter(noun_tokens)
word_freq.most_common(50)

[('year', 3906),
 ('time', 3567),
 ('day', 2973),
 ('life', 2516),
 ('mother', 2285),
 ('friend', 2214),
 ('man', 2063),
 ('way', 1914),
 ('child', 1762),
 ('thing', 1675),
 ('father', 1632),
 ('night', 1580),
 ('husband', 1508),
 ('people', 1450),
 ('woman', 1437),
 ('month', 1437),
 ('family', 1426),
 ('week', 1152),
 ('hand', 1129),
 ('parent', 1075),
 ('room', 1052),
 ('marriage', 1038),
 ('house', 972),
 ('eye', 934),
 ('school', 928),
 ('hour', 862),
 ('home', 820),
 ('date', 798),
 ('story', 772),
 ('bed', 767),
 ('word', 765),
 ('son', 742),
 ('person', 740),
 ('world', 719),
 ('daughter', 718),
 ('wife', 708),
 ('girl', 702),
 ('place', 700),
 ('moment', 689),
 ('face', 664),
 ('new', 664),
 ('boy', 658),
 ('guy', 658),
 ('morning', 656),
 ('car', 650),
 ('apartment', 647),
 ('boyfriend', 643),
 ('wedding', 635),
 ('heart', 633),
 ('head', 617)]

In [161]:
# least common words
word_freq.most_common()[-50:]

[('phillip', 1),
 ('sattler', 1),
 ('gabriella', 1),
 ('etoniru', 1),
 ('burbank', 1),
 ('latimer', 1),
 ('colleen', 1),
 ('goodhue', 1),
 ('marla', 1),
 ('hancock', 1),
 ('sheafs', 1),
 ('abdoh', 1),
 ('juliana', 1),
 ('perrett', 1),
 ('welton', 1),
 ('mackey', 1),
 ('oktoberfest', 1),
 ('lederhosen', 1),
 ('leach', 1),
 ('talula', 1),
 ('rena', 1),
 ('silverman', 1),
 ('marcelle', 1),
 ('beaulieu', 1),
 ('fuji', 1),
 ('platner', 1),
 ('leggett', 1),
 ('bbr', 1),
 ('pomales', 1),
 ('alastair', 1),
 ('mccready', 1),
 ('meagan', 1),
 ('fratiello', 1),
 ('okura', 1),
 ('cootner', 1),
 ('bitmoji', 1),
 ('clifford', 1),
 ('liles', 1),
 ('eicher', 1),
 ('nolen', 1),
 ('walston', 1),
 ('sundae', 1),
 ('commissioner', 1),
 ('kristine', 1),
 ('murawski', 1),
 ('reverend', 1),
 ('mouracade', 1),
 ('fawcett', 1),
 ('keavney', 1),
 ('lader', 1)]

### spaCy pipeline

Create custom factory to add to spacy pipeline to remove entities whose labels are 'PERSON'

In [76]:
#@Language.component('person_removal')
#def person_removal(doc):
#    ents = list(doc.ents)
   # for ent in ents:
   #     if ent.label_ == 'PERSON':
    #        ents.remove(ent)
   # ents = tuple(ents)
  #  doc.ents = ents
   # return (doc)
#Language.component("person_removal", func=person_removal)

In [77]:
#nlp.add_pipe('person_removal')

In [78]:
#print(nlp.pipe_names)

In [112]:
modern_love_df_rev[['essay','essay_no_stopwords']].head(2)

essay  \
0  Two days before I left South Dakota, Rex and I sat talking beneath the open hatchback of my car. In the distance, a lightning storm moved toward us over the open expanse of the Great Plains, churning the sky a murky purple. He was talking passionately about lithium batteries. The more he talked, the less he and I seemed to have in common. I considered myself someone who could become interested in almost anything, especially when I was attracted to the person speaking. But now I asked myself: Did I care about batteries? He and I were volunteers on the Pine Ridge Reservation, building and repairing infrastructure. He had been the first person to greet me when I arrived at the end of the long, dirt road. When he climbed out of the cab of a skid loader, and I saw his face, my body warmed. In the lyrical version of what happened as the storm approached, we would have stopped talking and taken seriously the pleasure of our bodies. But wanting to have a baby had made dating in my late 30s less like a poem and more like a math problem. There was a lot that had to line up, and what I was looking for now differed from what my younger self had envisioned. I didn’t care about dating someone for a certain amount of time before we had a baby, or being in love, or getting married. I wanted to like the biological father of my child, maybe admire him. That was about it. I had arrived at this set of criteria because the alternatives seemed sentimental and unrealistic, especially the husband-to-be wish lists that many of us champion during those years when we are both ready and able to have children. With the help of my sitting meditation practice, I had observed that the more I worried about getting pregnant, the less discerning I was about love, an effect I feared would intensify as I got older. How could I trust my judgment under pressure? Wouldn’t a lot of men start to smell like fathers? I decided the safest way to protect myself against romantic delusion would be to separate the two stories from the outset: I could try to find a mate or become a mother, but not at the same time. Since biological constraints made it easy to figure out which was more urgent, I resolved to have a child outside of the context of love. My solo road trip to South Dakota was conceived as an experience my future self, the one saddled with a dependent, would someday thank me for. When I returned home, I planned to get pregnant using an anonymous donor’s sperm. On my last evening with Rex, kissing in his tent, I realized there was a lot about him I didn’t know — who was in his life, where he worked, his last name. Before I crawled out of his tent, he asked for my phone number. He was headed home to Michigan, and I to California. I told him I thought we should leave things exactly as they were, which seemed perfect to me. “What, are you crazy?” he said, and he gave me his number. Back home, I pored over donor questionnaires at the local sperm bank, trying to keep straight who liked video games and who preferred billiards, but it all mixed blandly together for me. Phone conversations with Rex, though, were weird and memorable. He had inherited his father’s expressions such as “Son of a biscuit!” and “Jeez O’Pete’s!” Doting on his backyard laying hens, he often referred to himself as a “chicken mama.” He was the only 30-something adult I knew who had traveled on an airplane exactly once, a domestic round trip for a former job. We didn’t talk much about the parts of our lives that existed beyond the present. He mentioned that his relationship with a woman in Michigan was crumbling. All he knew of my path to motherhood was that I wanted a child. When my search for a donor stagnated from lacking a warm feeling about any of them, friends offered to screen profiles with me on the eve of my 40th birthday. Two donors received my friends’ approval, so I put myself on the wait-list for their sperm, though I still felt ambivalent. When I finally told Rex about my sta

Add some words to the stop words list to be removed from the corpus

In [87]:
sw_spacy = nlp.Defaults.stop_words

# verbs and a noun from top 50 tokens to remove
top_tokens_to_remove = {'relationship', 'say', 'love', 'know', 'like', 'want', 'feel', 'think', 'tell', 'go', 'ask', 'look','come'}
sw_spacy.update(top_tokens_to_remove)

# bottom 20 tokens to remove

twenty_least_common = word_freq.most_common()[-20:]
bottom_tokens_to_remove = set([a_tuple[0] for a_tuple in twenty_least_common])
sw_spacy.update(bottom_tokens_to_remove)

Apply updated stop words list to tokens

In [91]:
docs_clean_2 = [[w for w in doc if w not in sw_spacy] for doc in docs_clean]
modern_love_df_rev['essay_fewer_stopwords'] = docs_clean_2

<ipython-input-91-d9a1a52d80ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_fewer_stopwords'] = docs_clean_2


In [93]:
modern_love_df_rev[['essay_clean','essay_fewer_stopwords']].head(2)

,essay_clean,essay_fewer_stopwords
0,"[day, leave, south, dakota, rex, sit, talk, beneath, open, hatchback, car, distance, lightning, storm, move, open, expanse, great, plains, churn, sky, murky, purple, talk, passionately, lithium, battery, talk, common, consider, interested, especially, attract, person, speak, ask, care, battery, volunteer, pine, ridge, reservation, build, repair, infrastructure, person, greet, arrive, end, long, dirt, road, climb, cab, skid, loader, see, face, body, warm, lyrical, version, happen, storm, approach, stop, talk, take, seriously, pleasure, body, want, baby, date, late, like, poem, like, math, problem, lot, line, look, differ, young, self, envision, care, date, certain, time, baby, love, getting, marry, want, like, biological, father, child, ...]","[day, leave, south, dakota, rex, sit, talk, beneath, open, hatchback, car, distance, lightning, storm, open, expanse, great, plains, churn, sky, murky, purple, talk, passionately, lithium, battery, talk, common, consider, interested, especially, attract, person, speak, care, battery, volunteer, pine, ridge, reservation, build, repair, infrastructure, person, greet, arrive, end, long, dirt, road, climb, cab, skid, loader, face, body, warm, lyrical, version, happen, storm, approach, stop, talk, seriously, pleasure, body, baby, date, late, poem, math, problem, lot, line, differ, young, self, envision, care, date, certain, time, baby, getting, marry, biological, father, child, maybe, admire, arrive, set, criterion, alternative, sentimental, unrealistic, especially, husband, wish, ...]"
2,"[peter, hitch, wagon, young, early, wildly, handsome, gay, soulful, brown, eye, head, loose, unruly, curl, raise, african, american, woman, white, preacher, man, peter, spend, childhood, steep, evangelical, ministry, terrified, father, sermon, soothe, mother, inclination, administer, human, aspect, congregation, lead, night, meet, peter, fall, way, love, kismet, laugh, word, time, nuyorican, poets, cafe, premier, debut, joint, performance, poet, try, new, york, city, poetry, scene, read, piece, open, mic, slam, peter, admirer, way, applauding, crowd, tell, simultaneously, embody, fury, father, vulnerability, mother, destine, true, love, ask, fate, fix, say, god, leave, room, surprise, time, meet, fall, peter, parent, long, divorced, father, leave, ...]","[peter, hitch, wagon, young, early, wildly, handsome, gay, soulful, brown, eye, head, loose, unruly, curl, raise, african, american, woman, white, preacher, man, peter, spend, childhood, steep, evangelical, ministry, terrified, father, sermon, soothe, mother, inclination, administer, human, aspect, congregation, lead, night, meet, peter, fall, way, kismet, laugh, word, time, nuyorican, poets, cafe, premier, debut, joint, performance, poet, try, new, york, city, poetry, scene, read, piece, open, mic, slam, peter, admirer, way, applauding, crowd, simultaneously, embody, fury, father, vulnerability, mother, destine, true, fate, fix, god, leave, room, surprise, time, meet, fall, peter, parent, long, divorced, father, leave, church, peter, explore, truth, believe, ...]"


The below cell did run

In [33]:
# tokenization into words and parts of speech tagging

#from nltk.tokenize import word_tokenize, sent_tokenize
#from nltk.tag import pos_tag

#tokens = [pos_tag(word_tokenize(row)) for row in modern_love_df['essay']] # labels each word as a part of speech

The below cell ran but took a while to do so

In [ ]:
# named entity recognition

#from nltk.chunk import ne_chunk

#entities = [ne_chunk(token) for token in tokens] # this extracts entities from the list of words
#entities.draw()

The below cell did run

In [22]:
# Import module
#from nltk.tokenize import RegexpTokenizer
#from nltk.tag import pos_tag

# Create an instance of RegexpTokenizer for alphanumeric tokens
#tokeniser = RegexpTokenizer(r'\w+')

# Tokenize text and tag parts of speech
#tokens = [pos_tag(tokeniser.tokenize(row)) for row in modern_love_df['essay']] # labels each word as a part of speech

The below cell didn't work for me

In [40]:
# function to remove tuples in tokens that contain 'NNP' or 'NNPS'

#def remove_items(lst, item):
    #r = []
    #for i in lst:
       # if isinstance(i, list):
        #    r.append(remove_items(i, item))
       # elif item not in i:
         #   r.append(i)
   # return r

In [41]:
# remove proper nouns from tokenized text
#tokens_rev = remove_items(tokens,'NNP')
#tokens_rev = remove_items(tokens,'NNPS')

The below cell didn't work for me

In [43]:
#def filter_nnp_nnps(x):
  #  return [(u,v) for (u,v) in x if (v !='NNP') | (v != 'NNPS')]

#tokens_rev = [filter_nnp_nnps(sub_list) for sub_list in tokens]

The below cell works

In [30]:
# remove numbers and punctuation in essay column
    # used this resource: https://datagy.io/python-remove-punctuation-from-string/

#alphanumeric = lambda x: re.sub('[^a-zA-Z]', ' ', str(x)) # removes numbers
#punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower()) # removes punctuation and makes all letters lowercase

#modern_love_df_rev['essay_rev'] = modern_love_df_rev.essay.map(alphanumeric).map(punc_lower)

In [31]:
# the below code and text came from here: https://github.com/LucDemortier/MachineLearningWithTweets/blob/master/YogaTweetAnalysis.ipynb
'''
Load NLTK's English stopwords as well as its stemmer, then define:
* tokenize_and_stem: to tokenize into sentences, then words, 
  and return the stems of the words;
* tokenize_only: to only tokenize

Also create a stemmed version of the list of stopwords.  Note that the order 
of operations matters: (filter,stem) is not the same as (stem,filter), where "filter"
means "remove stopwords" in the first case and "remove stemmed stopwords" in the 
second.  For example, the word "willing" is not a stopword, but its stem "will" is.  
Thus, the order (stem,filter) tends to eliminate more tokens than (filter,stem).

Finally, create a dictionary that allows us to go back from stem to word (a one-to-many 
relationship that we arbitrarily reduce to a one-to-one relationship to facilitate 
interpretation of results later.)
'''
#import nltk
#from   nltk.stem.snowball import SnowballStemmer
#from   more_itertools import unique_everseen

#stopwords = nltk.corpus.stopwords.words('english')
#stopwords.append("'ve")
#stopwords.append("'re")
#more_stopwords = ["'d", 'could', 'might', 'must', "n't", 'need', 'r', 'sha', 'v', 'wo', 'would']
#stopwords.extend(more_stopwords)
#print('Stop words (%i): \n%s\n' %(len(stopwords),[str(word) for word in stopwords]))

#stemmer           = SnowballStemmer("english")
#stemmed_stopwords = list(unique_everseen([stemmer.stem(word) for word in stopwords]))
#print('Stemmed stop words (%i): \n%s\n' %(len(stemmed_stopwords),[str(word) for word in stemmed_stopwords]))

#alphanumeric = lambda x: re.sub('[^a-zA-Z]', ' ', str(x)) # removes numbers
#punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower()) # removes punctuation and makes all letters lowercase

#def tokenize_and_stem(text):
    
    #tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
   # stems = [stemmer.stem(t) for t in tokens]
    #return stems

#def tokenize_only(text):

  #  tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
   # return tokens

'\nLoad NLTK\'s English stopwords as well as its stemmer, then define:\n* tokenize_and_stem: to tokenize into sentences, then words, \n  and return the stems of the words;\n* tokenize_only: to only tokenize\n\nAlso create a stemmed version of the list of stopwords.  Note that the order \nof operations matters: (filter,stem) is not the same as (stem,filter), where "filter"\nmeans "remove stopwords" in the first case and "remove stemmed stopwords" in the \nsecond.  For example, the word "willing" is not a stopword, but its stem "will" is.  \nThus, the order (stem,filter) tends to eliminate more tokens than (filter,stem).\n\nFinally, create a dictionary that allows us to go back from stem to word (a one-to-many \nrelationship that we arbitrarily reduce to a one-to-one relationship to facilitate \ninterpretation of results later.)\n'

# Document similiarity: vectorize

In [100]:
# define corpus

docs_list_clean = [' '.join(doc) for doc in docs_clean_2]
corpus = docs_list_clean

In [101]:
'''
Apply tfidf transformation
max_df = maximum fraction of documents containing the word, for the word to be included in tfidf matrix
min_df = minimum fraction ...
ngram_range: We'll look at unigrams, bigrams and trigrams.
'''

vec_tfidf = TfidfVectorizer(max_df=0.80, min_df=0.01, ngram_range=(1,3))
X_tfidf = vec_tfidf.fit_transform(corpus).toarray()
pd.DataFrame(X_tfidf, columns=vec_tfidf.get_feature_names())

,abandon,abide,ability,able,abroad,abruptly,absence,absent,absolutely,absorb,...,young,young brother,young child,young girl,young man,young sister,young woman,youth,zone,zoom
0,0.000000,0.0,0.000000,0.050780,0.0,0.0,0.039439,0.0,0.000000,0.0,...,0.022051,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.040593,0.0,0.037210,0.026029,0.0,0.0,0.000000,0.0,0.043847,0.0,...,0.067819,0.050952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.058661,0.066108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.038088,0.0,0.034913,0.073267,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1500,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.125728,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1501,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1502,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Make a baseline topic model

The below cell works

In [18]:
# read in custom stop words
#with open('stop_words_english.txt', 'r') as f:
    #stopwords_custom = [s.strip() for s in f.readlines()]

The below cell works

In [19]:
# create function to lemmatize tokens and eliminate stopwords
#def lemmatize_and_eliminate(text):
    #lemmatizer = WordNetLemmatizer()
    #text = [word for word in text.split() if word not in stopwords_custom]
    #text = [lemmatizer.lemmatize(word) for word in text]
    #return " ".join(text)

The below cell works

In [26]:
# lemmatize and remove stop words from corpus
#corpus = modern_love_df_rev.essay_rev.map(lambda x: lemmatize_and_eliminate(x))

In [29]:
i = 600

doc = (pd.Series(name=corpus[i],
                data=X_tfidf[i], 
                index=vec_tfidf.get_feature_names()) 
        .sort_values(ascending=False))

doc.head()

linda       0.534776
accident    0.197141
motel       0.145908
car         0.120504
bowel       0.119363
Name: thursday morning cooking breakfast pajama daughter madeleine helping transferring egg cardboard carton refrigerator egg box pressing button elevator licking cake spoon morning performing duty sleepy devotion whisk asked madeleine throw egg peered curiously work egg nodded break throw madeleine ll catch looked egg small hand reason began carefully inspect shell trifling culinary request egg mixture unscripted moment trust father daughter wanted trust threw delicate save mother died year earlier sudden undiagnosed disorder suspected case marfan syndrome people heard madeleine talk day private nod listen ll share don answer time subtle exchange occurred week making dinner noticed madeleine leafing fashion magazine evening ve marry choked retrieved magazine opened featuring catwalk model blue dress yellow cape circled blue crayon bit snow white answer wasn time disney princess pla

# Principal Component Analysis

In [ ]:
# 
pca = PCA()